In [2]:
pip install opencv-python numpy pdf2image pillow pytesseract


In [3]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import os
from PIL import Image

# def preprocess_image(image):
#     """
#     Convert a PDF page image to a clean black-on-white version:
#     - Keeps black/dark text unchanged.
#     - Changes beige background to pure white.
#     """

#     # Convert image to NumPy array for OpenCV processing
#     img = np.array(image)

#     # Convert to HSV color space
#     hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

#     # Define HSV range for beige background (adjustable)
#     lower_beige = np.array([10, 20, 100], dtype=np.uint8)
#     upper_beige = np.array([40, 100, 255], dtype=np.uint8)

#     # Create a mask for the beige background
#     mask = cv2.inRange(hsv, lower_beige, upper_beige)

#     # Replace beige areas with white (255,255,255)
#     img[mask > 0] = [255, 255, 255]

#     # Convert back to PIL Image
#     return Image.fromarray(img)

# def preprocess_image(image):
#     """
#     Convert a PDF page image to a clean black-on-white version:
#     - Keeps black/dark text unchanged
#     - Changes beige background to pure white
#     - Uses adaptive thresholding for better text clarity
#     """
#     # Convert image to NumPy array for OpenCV processing
#     img = np.array(image)

#     # Convert to HSV color space and remove beige background
#     hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#     lower_beige = np.array([10, 20, 100], dtype=np.uint8)
#     upper_beige = np.array([40, 100, 255], dtype=np.uint8)
#     mask = cv2.inRange(hsv, lower_beige, upper_beige)
#     img[mask > 0] = [255, 255, 255]

#     # Convert to grayscale
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
#     # Apply adaptive thresholding
#     adaptive = cv2.adaptiveThreshold(
#         gray,
#         255,
#         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#         cv2.THRESH_BINARY,
#         blockSize=25,  # Controls the size of local region
#         C=5  # Controls threshold offset
#     )
    
#     # Blend original and adaptive threshold result
#     # 0.7:0.3 ratio maintains details while enhancing text clarity
#     result = cv2.addWeighted(gray, 0.7, adaptive, 0.3, 0)
    
#     # Convert back to RGB format
#     result = cv2.cvtColor(result, cv2.COLOR_GRAY2RGB)
    
#     return Image.fromarray(result)

def preprocess_image(image):
    img = np.array(image)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #lower_beige = np.array([5, 10, 80], dtype=np.uint8)   # Extended lower bound
    #upper_beige = np.array([45, 130, 255], dtype=np.uint8)  # Extended upper bound
    lower_beige = np.array([15, 15, 140], dtype=np.uint8)
    upper_beige = np.array([35, 80, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_beige, upper_beige)
    img[mask > 0] = [255, 255, 255]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    adjusted = cv2.convertScaleAbs(gray, alpha=1.4, beta=-20)
    result = cv2.cvtColor(adjusted, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(result)

def process_pdf(pdf_path, output_folder):
    """
    Extracts all pages from a PDF, cleans them by removing the beige background,
    and saves them as images in the specified folder.
    """
    # Open the PDF
    doc = fitz.open(pdf_path)

    # Create output folder if not exists
    os.makedirs(output_folder, exist_ok=True)

    for i in range(len(doc)):
        print(f"Processing page {i+1}...")

        # Render PDF page to an image
        page = doc[i]
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # Increase resolution
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Apply preprocessing (keep dark text, make background white)
        processed_img = preprocess_image(img)

        # Save image
        output_path = os.path.join(output_folder, f"page_{i+1:03d}.png")
        processed_img.save(output_path)

    print(f"✅ Processed {len(doc)} pages. Images saved in '{output_folder}'.")

# Example usage
pdf_path = "tome_12.pdf"  
output_folder = "BW_12"
process_pdf(pdf_path, output_folder)


Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44...
Processing page 45...
Processing page 46.

In [ ]:
# import pytesseract
# from PIL import Image
# import os

# # Set Tesseract OCR path (update if necessary)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# def perform_ocr_on_folder(image_folder, output_text_file, lang="fra"):
#     """
#     Runs OCR on all images in the specified folder and saves the extracted text.

#     :param image_folder: Folder containing processed images.
#     :param output_text_file: File where extracted text will be saved.
#     :param lang: OCR language (default: French - 'fra').
#     :return: None
#     """
#     extracted_text = ""

#     # Get all PNG images from the folder and sort them numerically
#     image_paths = sorted(
#         [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")]
#     )

#     if not image_paths:
#         print("❌ No images found in the folder!")
#         return

#     for image_path in image_paths:
#         print(f"📝 Performing OCR on {image_path}...")
#         text = pytesseract.image_to_string(Image.open(image_path), lang=lang)
#         extracted_text += f"\n\n=== Extracted Text from {os.path.basename(image_path)} ===\n\n{text}"
    
#     # Save extracted text to a file
#     with open(output_text_file, "w", encoding="utf-8") as f:
#         f.write(extracted_text)

#     print(f"✅ OCR completed! Extracted text saved to '{output_text_file}'.")

# # Example usage
# image_folder = "BW_12"  
# output_text_file = "extracted_text_12_BW.txt"

# perform_ocr_on_folder(image_folder, output_text_file, lang="fra")


📝 Performing OCR on BW_12\page_001.png...
📝 Performing OCR on BW_12\page_002.png...
📝 Performing OCR on BW_12\page_003.png...
📝 Performing OCR on BW_12\page_004.png...
📝 Performing OCR on BW_12\page_005.png...
📝 Performing OCR on BW_12\page_006.png...
📝 Performing OCR on BW_12\page_007.png...
📝 Performing OCR on BW_12\page_008.png...
📝 Performing OCR on BW_12\page_009.png...
📝 Performing OCR on BW_12\page_010.png...
📝 Performing OCR on BW_12\page_011.png...
📝 Performing OCR on BW_12\page_012.png...
📝 Performing OCR on BW_12\page_013.png...
📝 Performing OCR on BW_12\page_014.png...
📝 Performing OCR on BW_12\page_015.png...
📝 Performing OCR on BW_12\page_016.png...
📝 Performing OCR on BW_12\page_017.png...
📝 Performing OCR on BW_12\page_018.png...
📝 Performing OCR on BW_12\page_019.png...
📝 Performing OCR on BW_12\page_020.png...
📝 Performing OCR on BW_12\page_021.png...
📝 Performing OCR on BW_12\page_022.png...
📝 Performing OCR on BW_12\page_023.png...
📝 Performing OCR on BW_12\page_024